In [1]:
import sys
!{sys.executable} -m pip install "datasets==2.3.0" "transformers==4.20.1" "rouge-score"

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 361.9/361.9 KB 4.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 36.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.3/29.3 MB 51.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 15.5 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.7/123.7 KB 32.5 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.17.0
    Uninstalling transformers-4.17.0:
      Successfully uninstalled transformers-4.17.0


In [2]:
# Run to load all the Libraries and helper functions
%run "Utilities/libraries.py"

In [3]:
# INPUTS TO THE SCRIPTS
Bucket_Name="model-deploy-poc-new"

train_test_val_location_S3="train_test_val"


text_column="Article"
target_column="Summary"

In [4]:
sess=boto3.session.Session()
role=sagemaker.get_execution_role()

## 1. Data Loading and Pre-processing

In [5]:
# load data from S3 to Data
# prefix="BBC News Summary"
# !aws s3 cp "s3://{Bucket_Name}/{prefix}" "./Data/" --recursive

In [6]:
# Read data
dataset_path = Path('Data/')

articles_data = list(map(extract, dataset_path.glob('News Articles/*/*.txt')))
summaries_data = list(map(extract, dataset_path.glob('Summaries/*/*.txt')))

articles_df = pd.DataFrame(articles_data, columns=('Category', 'ID', text_column))
summaries_df = pd.DataFrame(summaries_data, columns=('Category', 'ID', target_column))

df = articles_df.merge(summaries_df, how='inner', on=('Category', 'ID'))

In [7]:
articles_df.shape

(2225, 3)

## 2. Data Cleaning 

In [8]:
df[text_column][0]

'Download chart debut is delayed\n\nThe inclusion of downloaded music in the official singles chart has been delayed for a month.\n\nThe British Phonographic Industry (BPI) had planned to include download formats in the chart from 20 March. But the date has been put back to 17 April to create a "level playing field" for independent labels. The BPI is concerned that independent repertoire is not adequately represented online and said they were looking at ways of rectifying it.\n\nBPI chairman Peter Jamieson said: "The inclusion of download formats in the Official UK Singles Chart is the most significant development in the charts for 20 years.\n\n"But for a multitude of reasons the current chart representation of independent repertoire at the major download outlets is poor. "I am very pleased we have secured this delay on behalf of independents. We have to try and establish a realistic \'level playing field\' of opportunity. The Official UK Charts are run by The Official UK Charts Compan

In [9]:
# We need to do some data cleaning before checking stats
df[text_column]=data_clean(df[text_column])
df[target_column]=data_clean(df[target_column])
df.drop(columns=['ID','Category'],inplace=True)
print(data_clean(df[text_column][:5])[0])


100%|██████████| 5/5 [00:00<00:00, 5556.84it/s]

download chart debut is delayed the inclusion of downloaded music in the official singles chart has been delayed for a month the british phonographic industry bpi had planned to include download formats in the chart from 20 march but the date has been put back to 17 april to create a level playing field for independent labels the bpi is concerned that independent repertoire is not adequately represented online and said they were looking at ways of rectifying it bpi chairman peter jamieson said the inclusion of download formats in the official uk singles chart is the most significant development in the charts for 20 years but for a multitude of reasons the current chart representation of independent repertoire at the major download outlets is poor i am very pleased we have secured this delay on behalf of independents we have to try and establish a realistic 'level playing field' of opportunity the official uk charts are run by the official uk charts company occ a joint venture between t

In [10]:
print((df[text_column].str.len()).describe(percentiles=[.25, .5, .75,0.85]))
print("#############################")
print((df[target_column].str.len()).describe(percentiles=[.25, .5, .75,0.85]))

count     2225.000000
mean      2207.062472
std       1330.001861
min        492.000000
25%       1412.000000
50%       1914.000000
75%       2739.000000
85%       3169.800000
max      24913.000000
Name: Article, dtype: float64
#############################
count     2225.000000
mean       984.697079
std        629.461568
min        225.000000
25%        613.000000
50%        851.000000
75%       1216.000000
85%       1424.400000
max      12213.000000
Name: Summary, dtype: float64


In [11]:
nth_decile_outlire_threshold=0.9

df=df[(df[text_column].str.len()<(df[text_column].str.len()).describe(percentiles=[nth_decile_outlire_threshold])[-2]) |
      (df[target_column].str.len()<(df[target_column].str.len()).describe(percentiles=[nth_decile_outlire_threshold])[-2])]

In [12]:
print((df[text_column].str.len()).describe(percentiles=[.25, .5, .75,0.85]))
print("#############################")
print((df[target_column].str.len()).describe(percentiles=[.25, .5, .75,0.85]))

count    2040.000000
mean     1952.873529
std       735.742038
min       492.000000
25%      1364.750000
50%      1818.500000
75%      2473.250000
85%      2844.300000
max      4125.000000
Name: Article, dtype: float64
#############################
count    2040.000000
mean      867.465686
std       334.565152
min       225.000000
25%       601.000000
50%       803.000000
75%      1106.000000
85%      1266.150000
max      1849.000000
Name: Summary, dtype: float64


## 3. Prepare data for the model

In [13]:
df_to_datadict=data_for_model_from_pandas(df,test_size=0.2,val_from_test_size=0.3)

In [14]:
df_to_datadict

DatasetDict({
    train: Dataset({
        features: ['Article', 'Summary', '__index_level_0__'],
        num_rows: 1632
    })
    test: Dataset({
        features: ['Article', 'Summary', '__index_level_0__'],
        num_rows: 326
    })
    val: Dataset({
        features: ['Article', 'Summary', '__index_level_0__'],
        num_rows: 82
    })
})

In [21]:
!pip install --upgrade fsspec

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [23]:
import datasets

In [29]:
import s3fs
fs = s3fs.S3FileSystem()

In [31]:
df_to_datadict.save_to_disk(f"s3://{Bucket_Name}/{train_test_val_location_S3}/", fs=fs)

Flattening the indices:   0%|          | 0/2 [00:00<?, ?ba/s]

Flattening the indices:   0%|          | 0/1 [00:00<?, ?ba/s]

Flattening the indices:   0%|          | 0/1 [00:00<?, ?ba/s]

In [32]:
input_path = f"s3://{Bucket_Name}/{train_test_val_location_S3}"
input_path

's3://model-deploy-poc-new/train_test_val'